## BME i9400
## Fall 2024
### Transformers and Large Language Models

## Goals of this Lecture

1. Understand the fundamentals of the transformer architecture.
2. Learn how the attention mechanism works and why it is central to transformers.
3. Gain intuition about why transformers are so effective.
4. Explore how transformers are used to build large language models through pretraining and fine-tuning.

## Some fantastic resources for understanding the Transformer architecture
- Wrapping your head around transformers will take multiple passes. However, it is well worth the effort!
- Some resources that I found particularly helpful:
    - [Transformer Explainer](https://poloclub.github.io/transformer-explainer/)
    - [Understanding self-attention](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)
    - [3 Blue 1 Brown video on transformers](https://www.youtube.com/watch?v=eMlx5fFNoYc) 

## The classic picture of the Transformer architecture
- This is the picture that appears in the original paper by Vaswani et al. (2017).
    - Vaswani, A. (2017). Attention is all you need. Advances in Neural Information Processing Systems.
- Despite appearing in the paper, it is not a good starting point for understanding transformers.
- In fact the original paper is not a good resource for learning the architecture.
- In the paper, transformers are mainly posed as a solution to machine translation: converting a sentence in one language to another.
<img src="transformer.webp" width="600" height="600">

## What is a Transformer?
- *Transformers* are a type of model architecture that has revolutionized natural language processing (NLP).
- Key innovation: Self-attention mechanism for processing sequences.
    - Replaced previous architectures that included convolutional neural nets (CNNs) models in NLP.
- Enabled state-of-the-art performance in:
    - Translation
    - Summarization
    - Question answering


## Transformer Architecture Overview
- Composed of an encoder and decoder
    - Encoder: Processes input sequences (i.e., English sentence)
	- Decoder: Generates output sequences (i.e., French translation)
- Core components:
	1.	Self-Attention: Captures relationships between words in a sequence.
	2.	Feedforward Layers: Nonlinear transformations for learning representations.
	3.	Positional Encoding: Adds order information to the sequence.
- Stacked layers of attention and feedforward modules allow deep learning.

## Why Attention?

- Attention allows the model to focus on relevant parts of the sequence.
- Example: In a sentence, “The cat sat on the mat,” the word “cat” is strongly related to “sat.”
- Attention weights capture these relationships.
- Self-attention captures long-range dependencies better than previous approaches.

## Tokenization, Embedding, Positional Encoding
- To start, we convert the input sentence into a sequence of tokens
- These tokens are initially represented by integers
- We then convert these integers into dense vectors called *embeddings*
- To capture the order of the tokens, we add *positional encodings* to the embeddings
    - Positional encoding is a sine-cosine function of the position of the token in the sequence
    - These sine-cosine functions are literally added to the embeddings
<img src="embedding.png" width="900">

## The Attention Mechanism
1.	Input: A sequence of word embeddings.
2.  Query, Key, Value (Q, K, V):
    - Represent each word with three vectors.
3. Attention Scores:
	    - Compute scores for each word using:
 
___
$ \text{Score}(Q, K) = \frac{Q \cdot K^\top}{\sqrt{d_k}} $ 
___
        
4.  Apply softmax to normalize scores.
5. Compute context vector as weighted summation of values:
	    - Combine values (V) weighted by attention scores.
**The context vector is the output of the attention mechanism.**


## Multi-Head Attention
-   Splits attention into multiple “heads” for diverse perspectives.
-   Each head independently computes attention, focusing on different relationships.
-   Outputs are concatenated and linearly transformed.

___ 
$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \text{head}_2, \ldots)W^O$

___ 
<img src="QKV.png" width="900">


## Masked Self-Attention
- Attention Score
    - The dot product between all query-key pairs is computed and represents how much the quary is related to the key.
- Masking
    - A "mask" is applied to the matrix of attention scores so that the model is not able to see the future words (this would be cheating in the context of language modeling).
- Softmax
    - Attention scores are converted into probabilities using the softmax function.
    - The resulting matrix elements represent how strongly each word relates to the words to its left. 
<img src="attention.png" width="900" >

## Output of self-attention block
- The (softmax-ed) self-attention scores are multiplied by the value matrix to produce the output of the self-attention block.

## Feedforward Layer
- The context vectors from the self-attention block are passed through a feedforward layer.
- The task of this layer is to exploit the context of the sentence in order to predict the next word.
- In the end, this results in a vector of probabilities, one for each token in the vocabulary.
- The token with the highest probability is the predicted next word.
<img src="mlp.png" width="900">

## Why Are Transformers So Effective?
-   Parallelization: Processes entire sequences simultaneously (vs. sequentially in RNNs).
-   Scalability: Handles very large datasets and model sizes.
-   Representation Power: Captures complex relationships with self-attention.
-   Versatility: Works across various tasks with minimal architecture changes.

## How Are LLMs Built?

-   Pretraining: train on massive corpora (e.g., books, biomedical papers).
-   Pretraining tasks:
    -   Masked language modeling (BERT).
	-   Autoregressive prediction (GPT).
-   Fine-Tuning:
    - Adapt pretrained models to specific tasks with smaller datasets (e.g., question answering).

## Transformers in Biomedical Engineering
-	Literature mining: Extract disease-gene associations from PubMed abstracts.
-   Clinical notes analysis: Summarize patient records or identify key trends.
-	Drug discovery: Predict drug-target interactions or design new molecules.

## Hands on demo with the Hugging Face Transformers library

In [ ]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load a pretrained masked language model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Encode a sentence with a masked token
text = "The heart is a [MASK] organ."
inputs = tokenizer(text, return_tensors="pt")

# Display the token IDs
print("Input IDs:", inputs.input_ids)

In [ ]:
# Generate the mode's prediction for the masked token
outputs = model(**inputs)

# Get predicted token for the masked position
predictions = outputs.logits
mask_index = inputs.input_ids[0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = predictions[0, mask_index].argmax(dim=-1).item()
print("Predicted Token:", tokenizer.decode([predicted_token_id]))

In [ ]:
# get the top 5 predictions
top_k = 5
top_predictions = predictions[0, mask_index].topk(top_k).indices
top_predictions = [tokenizer.decode([token_id]) for token_id in top_predictions]
print("Top Predictions:", top_predictions)

## Finally, what you've all been waiting for

## Install the openai API
```! pip install openai```

## Load the API key

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(dotenv_path="/Users/jacekdmochowski/.env")  # Adjust the path if your .env file is elsewhere

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

## Talk to the GPT-4o model

In [12]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert forecaster of the future."},
        {
            "role": "user",
            "content": "What will 2025 bring to the world?"
        }
    ]
)

In [11]:
response = completion.choices[0].message.content
# render with line breaks
print(response.replace("\n", "\n"))

Predicting the future with certainty is impossible, but based on current trends and data available up to October 2023, several developments and trends are likely to influence the world in 2025:

1. **Technology and Artificial Intelligence**: AI will continue to advance, becoming more integrated into everyday life and business. We may see significant improvements in AI-driven healthcare, autonomous vehicles, and personalized education tools.

2. **Climate Change and Environment**: Efforts to combat climate change will likely intensify. More countries may adopt aggressive policies to reduce carbon emissions, and renewable energy technologies, such as solar and wind, could see even wider adoption.

3. **Global Economy**: The global economy will continue to adapt to challenges brought by technological changes and geopolitical tensions. Digital currencies and blockchain technology might gain more traction, impacting traditional financial systems.

4. **Healthcare**: Advances in biotechnolog